<a href="https://colab.research.google.com/github/jainilparikh/NeuralArtTransfer/blob/master/neutalArtTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask
import numpy as np
import cv2
import tensorflow as tf;
import imageio
import scipy.misc
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
#from google.colab import files

#content_img = files.upload()

In [3]:
#from google.colab import files
#styleImage = files.upload()

In [4]:
def get_image(imgpath):
    w=512
    h=512
    loaded_img = tf.keras.preprocessing.image.load_img(imgpath,target_size=(w,h))
   
    image = tf.keras.applications.vgg19.preprocess_input(tf.keras.preprocessing.image.img_to_array(loaded_img))
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = np.expand_dims(image , axis = 0)
    return image
def imgshow(image):
    image[:,:,0]  *=64/255.0
    image[:,:,1] *=128/255.0
    image[:,:,2] *=192/255.0
    image = image[:,:,::-1]
    image = np.clip(image,0,255).astype('uint8')
    img=np.squeeze(image)
    plt.figure(figsize = (50,10))
    plt.imshow(img, interpolation='nearest')
def imshowImage(image):
  image[:,:,0]  *=64/255.0
  image[:,:,1] *=128/255.0
  image[:,:,2] *=192/255.0
  image = image[:,:,::-1]
  image = np.clip(image,0,255).astype('uint8')
  img=np.squeeze(image)
  return img
#styleImage = get_image("/content/style.jpg")
#contentImage = get_image("/content/content.jpg")
#imgshow(styleImage)


In [5]:
from tensorflow.keras.applications.vgg19 import VGG19

# Layer Extraction From VGG Model 

In [6]:
baseModel = VGG19(weights='imagenet',include_top=False,pooling='Max');
baseModel.trainable = False;
contentImageLayers = ['block5_conv2']
styleImageLayers = ["block1_conv1","block2_conv1","block3_conv1","block4_conv1","block5_conv1"]
baseModel.get_layer(contentImageLayers[0])
print(baseModel.input)

80142336/80134624 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


#Model Prep

In [7]:
contentModel = Model(inputs = baseModel.input,outputs = baseModel.get_layer(contentImageLayers[0]).output) 

In [8]:
styleModel = [Model(inputs = baseModel.input , outputs = baseModel.get_layer(Layer).output) for Layer in styleImageLayers]

# Content Loss

In [9]:
def customLoss(model,inputImage,outputImage):
  val = tf.reduce_mean((model(inputImage) - model(outputImage))**2)
  return val;

# GRAM Matrix ( For Style Loss)

In [10]:
def gramMatrix(inputImage):
  channels=int(inputImage.shape[-1])
  a=tf.reshape(inputImage,[-1,channels])
  n=tf.shape(a)[0]
  gram=tf.matmul(a,a,transpose_a=True)
  return(gram/tf.cast(n,tf.float32))

# Style Loss

In [11]:
def styleLoss(model,inputImage,generated):
  error = 0
  for styleModels in model:
    styleVal = styleModels(inputImage);
    generatedVal = styleModels(generated)
    val = tf.reduce_mean((gramMatrix(styleVal) - gramMatrix(generatedVal))**2);
    error = error + val*1/len(model);
  return error;

# Training the Initial Image with Content and Style Images

In [12]:
def runFunction(contentImage,styleImage):
  ## Calculate loss for content image
  ## Calculate loss for style image
  ## update the initial image by the error from the losses calculated
  ## repeat the process
  contentGenerated = tf.Variable(contentImage,dtype = tf.float32);
  optimizer = tf.keras.optimizers.Adam(learning_rate = 10)
  maxImage = contentGenerated;
  maxCost = 1e12

  ## Weighted Loss calcualtion
  for i in range(0,10):
    print('Iteration : ' + str(i))
    with tf.GradientTape() as tape:
      contentLossVal = customLoss(contentModel,contentImage,contentGenerated)
      styleLossVal = styleLoss(styleModel , styleImage , contentGenerated)
      cost = contentLossVal * 40 + styleLossVal * 1000;
      gradient = tape.gradient(cost , contentGenerated);
      optimizer.apply_gradients([(gradient,contentGenerated)])
      print(cost)
      if(cost < maxCost):
        maxCost = cost;
        maxImage = contentGenerated.numpy();
  return maxImage,contentGenerated.numpy();

In [13]:
#img,imgVal = runFunction(contentImage,styleImage)

In [14]:
## Original Image

#imgshow(contentImage)

# Result Image

In [15]:
#imgshow(imgVal)

In [16]:
!pip install flask-ngrok

In [ ]:
## FLASK
##  Hosted on ngrok :: http://f211c07033ea.ngrok.io
!pip install flask-ngrok
import io
from PIL import Image
import os
from flask import request, Response
from flask import render_template
from flask import send_file
from flask_ngrok import run_with_ngrok
from flask import Flask,flash
from werkzeug.utils import secure_filename
import urllib.request

from os.path import join, dirname, realpath

from PIL import Image
import base64
import io

#UPLOADS_PATH = join(dirname(realpath(__file__)), 'static/uploads/..')

app = Flask(__name__, template_folder='drive/My Drive/NeuralArtTransfer/templates',static_folder='drive/My Drive/NeuralArtTransfer/static')
UPLOAD_FOLDER = 'static/uploads/'
app.secret_key = "secret key"

app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
## The HTML Pages are stored in tempelate folder 
folder = "drive/My Drive/NeuralArtTransfer/templates";

## Direct folder where uploaded image will be saved
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)  ## Start ngork

ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/',methods = ['POST' , 'GET'])   ## First Page

## Sending Image to the server
def neuralArt():
  return render_template('imageupload.html')


@app.route('/imageInput' , methods = ['POST' ,'GET'])  ## Second Page
def getImage():
  if 'content' not in request.files or 'style' not in request.files:
    flash('No file part')
    return render_template('imageupload.html')
  content = request.files['content']
  style = request.files['style']
  if content.filename == '' or style.filename == '':
    flash('No image selected for uploading')
    return render_template('imageupload.html')
  if content and allowed_file(content.filename) and style and allowed_file(style.filename):
    
    #im = Image.open(content)
    #data = io.BytesIO()
    #im.save(data, "JPEG")
    #encoded_img_data = base64.b64encode(data.getvalue())
    content.filename = "content.jpg"
    style.filename = "style.jpg"
    filename1 = secure_filename(content.filename)
    #content.save(os.path.join(app.config['/static/uploads'], filename))
    content.save( filename1)
    #flash('upload_image filename: ' + filename)
    filename2 = secure_filename(style.filename)
    style.save( filename2)
    #return "<H1> Welcome to Neural Art Tranfer</H1>"

    #return render_template("index.html", img_data=encoded_img_data.decode('utf-8'))
  else:
    flash('Allowed image types are -> png, jpg, jpeg, gif')
    return render_template('imageupload.html')
    
  contentImage = get_image("content.jpg")
  styleImage = get_image("style.jpg")
  _ , resImage = runFunction(contentImage,styleImage)
  resImage = resImage.reshape((resImage.shape[1] , resImage.shape[2],resImage.shape[3]))


  #add other functionalities to convert resImage to resimg.jpg
  imageio.imwrite('/content/resImage.jpg', resImage)
  content_type = 'image/jpeg'
  file_object = io.BytesIO()
  img = imshowImage(resImage);
  img = Image.fromarray(img.astype('uint8'))
  img.save(file_object, 'jpeg')
  file_object.seek(0)
  return send_file(file_object , mimetype = 'image/jpeg')


  return #render_template("output.html", content = "/content/content.jpg",style="/content/style.jpg",result="drive/My Drive/NeuralArtTransfer/static/resImage.jpg")


if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c0da8c0f6c66.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Mar/2021 16:00:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2021 16:00:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/js/scripts.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/js/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/js/jquery-3.3.1.slim.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/js/jquery-3.3.1.slim.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2021 16:00:55] "GET /static/js/bootstrap.bundle.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - -

Iteration : 0
tf.Tensor(16325399000000.0, shape=(), dtype=float32)
Iteration : 1
tf.Tensor(9247469000000.0, shape=(), dtype=float32)
Iteration : 2
tf.Tensor(7195081600000.0, shape=(), dtype=float32)
Iteration : 3
tf.Tensor(6005105600000.0, shape=(), dtype=float32)
Iteration : 4
tf.Tensor(5548733000000.0, shape=(), dtype=float32)
Iteration : 5
tf.Tensor(3256712200000.0, shape=(), dtype=float32)
Iteration : 6
tf.Tensor(3038176300000.0, shape=(), dtype=float32)
Iteration : 7
tf.Tensor(2375224400000.0, shape=(), dtype=float32)
Iteration : 8
tf.Tensor(2294355600000.0, shape=(), dtype=float32)
Iteration : 9


127.0.0.1 - - [17/Mar/2021 16:02:02] "POST /imageInput HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2021 16:02:02] "POST /imageInput HTTP/1.1" 200 -


tf.Tensor(1821734100000.0, shape=(), dtype=float32)


## APP RELATED CODE IGNORE

In [22]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
imgVal = get_image("/content/templates/style.jpg")

In [ ]:
imgVal.astype('uint8')

In [ ]:
imgVal = get_image("/content/templates/style.jpg")
imgshow(imgVal)
print(imgVal.shape)
imgVal = imgVal.reshape((imgVal.shape[1] , imgVal.shape[2] , imgVal.shape[3] ))
imgVal = Image.fromarray(imgVal.astype('uint8'))
imgVal.save('COntenetIageas.jpg')

In [ ]:
cv2.imwrite("COntentImage.jpg",imgVal);

In [ ]:
## 
## 1) Get the ImageOutput Function to work to save an image in colab
## 2) Send Image from colab to app
## 3) Display Image
## 4) Solve Preview Image prob on App

In [ ]:
contentImage = get_image("content.jpg")
styleImage = get_image("style.jpg")
_ , resImage = runFunction(contentImage,styleImage)

In [ ]:
print(resImage.shape[3])
resImage = resImage.reshape((resImage.shape[1] , resImage.shape[2],resImage.shape[3]))

In [ ]:
  imageio.imwrite('resImage.jpg', resImage)